In [1]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA,TruncatedSVD,NMF
from sklearn.preprocessing import Normalizer
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
matplotlib.style.use('ggplot')
pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.options.mode.chained_assignment = None

In [2]:
constituents=pd.read_csv("/data/dharp/compounding/datasets/words.csv",nrows=1_000_000)
constituents.columns=['joiner','context','decade','count']
constituents=constituents.query('decade != 2000')
constituents=constituents.groupby(['joiner','decade','context'])['count'].sum()
display(constituents.shape)
constituents.head()

(924870,)

joiner  decade  context
a_n     1800    a_a         15
                a_n        128
                aa_n        20
                aaa_n       16
                aaron_n      6
Name: count, dtype: int64

In [3]:
compounds=pd.read_csv("/data/dharp/compounding/datasets/phrases.csv",nrows=1_000_000)
compounds.columns=['modifier','head','context','decade','count']
compounds=compounds.query('decade != 2000')
compounds['joiner']=compounds['modifier']+" "+compounds['head']
compounds=compounds.groupby(['joiner','decade','context'])['count'].sum()
display(compounds.shape)
compounds.head()

(850316,)

joiner    decade  context
a_n aa_n  1820    a_n        1
                  aa_n       1
          1870    a_n        1
                  aa_n       1
          1920    a_n        1
Name: count, dtype: int64

In [4]:
df=pd.concat([constituents,compounds])

In [5]:
df=df.to_sparse()

In [6]:
df, rows, columns = df.to_coo(row_levels=['joiner','decade'],column_levels=['context'],sort_labels=False)

In [7]:
len(columns)

39573

In [8]:
df_svd = TruncatedSVD(n_components=300, n_iter=10, random_state=1991)
display(df_svd.fit(df).explained_variance_ratio_.sum())
df_list=df_svd.fit(df).explained_variance_ratio_
df_reduced = df_svd.fit_transform(df)
df_reduced = Normalizer(copy=False).fit_transform(df_reduced)
df_reduced=pd.DataFrame(df_reduced,index=rows)
#df_reduced.reset_index(inplace=True)
df_reduced.shape

0.9999994049311242

(337167, 300)

In [9]:
df_reduced.head()

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
"(a_n, 1800)",0.957,-0.225,-0.104,-0.002,0.120,-0.033,0.037,-0.001,0.020,-0.006,...,0.000,0.000,-0.000,0.000,-0.000,0.000,-0.000,0.000,-0.000,-0.000
"(a_n, 1810)",0.960,-0.225,-0.100,-0.003,0.111,-0.033,0.033,-0.001,0.020,-0.005,...,-0.000,-0.000,0.000,-0.000,0.000,0.000,0.000,0.000,-0.000,0.000
"(a_n, 1820)",0.960,-0.228,-0.103,-0.005,0.106,-0.029,0.028,-0.000,0.013,-0.004,...,-0.000,-0.000,0.000,-0.000,0.000,0.000,0.000,-0.000,0.000,0.000
"(a_n, 1830)",0.965,-0.221,-0.089,-0.009,0.099,-0.035,0.028,-0.000,0.017,-0.001,...,0.000,0.000,0.000,0.000,-0.000,-0.000,0.000,-0.000,-0.000,-0.000
"(a_n, 1840)",0.967,-0.219,-0.084,-0.007,0.087,-0.031,0.022,-0.000,0.012,0.000,...,0.000,0.000,-0.000,0.000,-0.000,0.000,-0.000,0.000,-0.000,0.000


In [10]:
df_reduced.index = pd.MultiIndex.from_tuples(df_reduced.index, names=['joiner', 'decade'])

In [12]:
co_occ_reduced.reset_index(inplace=True)

In [13]:
    compounds_reduced=df_reduced.loc[~df_reduced.index.get_level_values(0).str.contains(r'\w \w')]
    #compounds_reduced.reset_index(inplace=True)
    compounds_reduced

0      1      2      3      4      5      6      7    \
joiner         decade                                                          
a_n            1800   0.957 -0.225 -0.104 -0.002  0.120 -0.033  0.037 -0.001   
               1810   0.960 -0.225 -0.100 -0.003  0.111 -0.033  0.033 -0.001   
               1820   0.960 -0.228 -0.103 -0.005  0.106 -0.029  0.028 -0.000   
               1830   0.965 -0.221 -0.089 -0.009  0.099 -0.035  0.028 -0.000   
               1840   0.967 -0.219 -0.084 -0.007  0.087 -0.031  0.022 -0.000   
               1850   0.969 -0.218 -0.084 -0.007  0.074 -0.023  0.016  0.000   
               1860   0.972 -0.214 -0.082 -0.007  0.052 -0.009  0.007  0.000   
               1870   0.975 -0.205 -0.071 -0.009  0.036 -0.001 -0.002 -0.000   
               1880   0.977 -0.199 -0.065 -0.007  0.023  0.006 -0.006 -0.000   
               1890   0.980 -0.191 -0.060 -0.007  0.007  0.011 -0.010 -0.000   
               1900   0.982 -0.181 -0.050 -0.005 -0.003  0.013 -0.007 -0.000   
               1910   0.988 -0.154 -0.020 -0.004 -0.022  0.009 -0.002  0.000   
               1920   0.990 -0.141 -0.013  0.000 -0.025  0.004  0.002  0.000   
               1930   0.993 -0.116 -0.004  0.006 -0.025 -0.002  0.014  0.000   
               1940   0.995 -0.089  0.008  0.008 -0.026 -0.007  0.019 -0.000   
               1950   0.997 -0.074  0.023  0.005 -0.014 -0.008  0.007 -0.000   
               1960   0.998 -0.055  0.040  0.002 -0.001 -0.008 -0.006 -0.000   
               1970   0.999 -0.013  0.038  0.002  0.007  0.000 -0.001  0.000   
               1980   1.000  0.023  0.014  0.001  0.005  0.008  0.003 -0.000   
               1990   0.999  0.039 -0.019 -0.002 -0.002 -0.002 -0.001  0.000   
aa_n           1800   0.015 -0.002  0.004 -0.001  0.002  0.005  0.000 -0.000   
               1810   0.004  0.003  0.008  0.001  0.012  0.020  0.011 -0.001   
               1820   0.050 -0.009  0.005 -0.004  0.003  0.024  0.008 -0.001   
               1830   0.031 -0.006  0.002 -0.002  0.007  0.018  0.010 -0.001   
               1840   0.154 -0.041 -0.003 -0.014 -0.014  0.029 -0.002 -0.001   
               1850   0.273 -0.045  0.028 -0.014 -0.023  0.024 -0.033 -0.000   
               1860   0.021 -0.003  0.004 -0.001  0.000  0.004 -0.001 -0.000   
               1870   0.472 -0.069  0.062 -0.034 -0.021  0.040 -0.063 -0.001   
               1880   0.152 -0.024  0.019 -0.011 -0.010  0.014 -0.017 -0.000   
               1890   0.775 -0.120  0.094 -0.060 -0.039  0.048 -0.119 -0.001   
...                     ...    ...    ...    ...    ...    ...    ...    ...   
abolitionism_n 1900   0.817 -0.173 -0.159  0.082  0.145 -0.074  0.058 -0.001   
               1910   0.871 -0.180 -0.056  0.064  0.105 -0.030  0.003 -0.001   
               1920   0.838 -0.125 -0.060  0.030  0.073  0.002  0.035 -0.001   
               1930   0.205 -0.032 -0.003  0.002  0.037 -0.008 -0.000  0.002   
               1940   0.582 -0.131 -0.088  0.051  0.129 -0.078  0.049  0.001   
               1950   0.771 -0.118 -0.033 -0.034  0.029 -0.005 -0.066 -0.000   
               1960   0.555 -0.101  0.024 -0.021  0.038 -0.013 -0.046  0.001   
               1970   0.075 -0.013  0.004  0.002 -0.002 -0.001 -0.009  0.001   
               1980   0.056 -0.001 -0.002  0.001 -0.003 -0.017 -0.022  0.001   
               1990   0.069  0.028  0.002  0.005  0.007 -0.022 -0.027  0.001   
abolitionist_n 1800   0.085 -0.004  0.012 -0.005  0.010  0.002 -0.020 -0.000   
               1810   0.222 -0.016 -0.057  0.042  0.005  0.018  0.034 -0.001   
               1820   0.810 -0.098  0.066 -0.062 -0.039  0.056 -0.119 -0.001   
               1830   0.805 -0.030 -0.195  0.175  0.029  0.042  0.003 -0.003   
               1840   0.800 -0.042 -0.252  0.202  0.005  0.077  0.148 -0.003   
               1850   0.894 -0.077 -0.135  0.102 -0.021  0.066  0.048 -0.002   
               1860   0.923 -0.086 -0.081  0.070 -0.032  0.062  0.004 -0.002   
               1870   0.

In [16]:
modifiers=co_occ_reduced.loc[co_occ_reduced.constituent.str.endswith(r'_m')]
modifiers['constituent']=modifiers['constituent'].str.replace(r'_m$', r'_n', regex=True)
modifiers.to_csv('/data/dharp/compounding/datasets/modifiers_context_decadal_svd.csv',index=False,header=False,sep='\t')